In [1]:
import pickle

with open('C:\\Users\\wangyi66\\Desktop\\AVS\\video_3way.pickle', 'rb') as openfile:
    try:
        objects = pickle._Unpickler(openfile)
        objects.encoding = 'latin1'
        A = objects.load()
        print(len(A))
        print(len(A[0]))
        print(A[1].shape)
    except EOFError:
        print('failed')


13
2250
(2250, 98, 3)


## ALL above is inrelevant code

In [1]:
import sys
sys.path.append('C:\\Users\\wangyi66\\Desktop\\cam\\Year_four\\CMU-MultimodalSDK-Tutorials')
from constants import SDK_PATH, DATA_PATH, WORD_EMB_PATH, CACHE_PATH

if SDK_PATH is None:
    print("SDK path is not specified! Please specify first in constants/paths.py")
    exit(0)
else:
    sys.path.append(SDK_PATH)

import mmsdk
import os
import re
import numpy as np
from mmsdk import mmdatasdk as md
from subprocess import check_call, CalledProcessError

# create folders for storing the data
if not os.path.exists(DATA_PATH):
    check_call(' '.join(['mkdir', '-p', DATA_PATH]), shell=True)

In [2]:
D_PATH = 'D:\\CMU-MOSEI_Poria\\MOSEI'

In [3]:
import os
import numpy as np
audio_path = os.path.join(D_PATH, 'audio.npz')
audio = np.load(audio_path, mmap_mode='r')
text_path = os.path.join(D_PATH, 'text.npz')
text = np.load(text_path, mmap_mode='r')
video_path = os.path.join(D_PATH, 'video.npz')
video = np.load(video_path, mmap_mode='r')

train_text    = text['train_data']
train_audio   = audio['train_data']
train_video   = video['train_data']
print('train_text', train_text.shape)

valid_text    = text['valid_data']
valid_audio   = audio['valid_data']
valid_video   = video['valid_data']
print('valid_text', valid_text.shape)

test_text     = text['test_data']
test_audio    = audio['test_data']
test_video    = video['test_data']
print('test_text', test_text.shape)

senti_train_label   = video['trainSentiLabel']
senti_valid_label   = video['validSentiLabel']
senti_test_label    = video['testSentiLabel']
print('senti_train_label', senti_train_label.shape)

#senti_train_label   = to_categorical(senti_train_label >= 0)
#senti_valid_label   = to_categorical(senti_valid_label >= 0)
#senti_test_label    = to_categorical(senti_test_label >= 0)

emo_train_label   = video['trainEmoLabel']
emo_valid_label   = video['validEmoLabel']
emo_test_label    = video['testEmoLabel']

train_length  = video['train_length']
valid_length  = video['valid_length']
test_length   = video['test_length']
print(train_length)

#max_segment_len = train_text.shape[1]
#print(max_segment_len)
#for i in range(2250 // 25):
#    print(train_length[i*25:(i+1) * 25])

train_text (2250, 98, 300)
valid_text (300, 98, 300)
test_text (678, 98, 300)
senti_train_label (2250, 98, 1)
[ 6  3 10 ...  5 13  1]


In [4]:
DATASET = md.cmu_mosei
all_num = DATASET.standard_folds.standard_train_fold + \
     DATASET.standard_folds.standard_test_fold + \
     DATASET.standard_folds.standard_valid_fold
print('all', len(all_num))
print('train', len(DATASET.standard_folds.standard_train_fold))
print('valid', len(DATASET.standard_folds.standard_valid_fold))
print('test', len(DATASET.standard_folds.standard_test_fold))


all 3227
train 2249
valid 300
test 678


In [5]:
from mmsdk.mmdatasdk import log, computational_sequence
import numpy
import time
import struct
from tqdm import tqdm_notebook as tqdm

In [6]:
# define your different modalities - refer to the filenames of the CSD files
basic_dict={'COVAREP': os.path.join(DATA_PATH, 'CMU_MOSEI_COVAREP.csd'), 
            'WordVec': os.path.join(DATA_PATH, 'CMU_MOSEI_TimestampedWordVectors.csd')}
second_dict = {'Facet': os.path.join(DATA_PATH, 'CMU_MOSEI_VisualFacet42.csd'),
            'OpenFace': os.path.join(DATA_PATH, 'CMU_MOSEI_VisualOpenFace2.csd')}
other_dict = {'Word': os.path.join(DATA_PATH, 'CMU_MOSEI_TimestampedWords.csd'),
             'Phone': os.path.join(DATA_PATH, 'CMU_MOSEI_TimestampedPhones.csd')}
label_dict = {'mylabels':os.path.join(DATA_PATH, 'CMU_MOSEI_Labels.csd')}

In [7]:
basic_dataset = md.mmdataset(basic_dict)
second_dataset = md.mmdataset(second_dict)
label_dataset = md.mmdataset(label_dict)

[2021-02-26 16:59:02.825] | Success | Computational sequence read from file D:\cmumosei\CMU_MOSEI_COVAREP.csd ...
[2021-02-26 16:59:30.024] | Warning | Validation of the computational sequence skipped by user request.
[2021-02-26 16:59:30.040] | Success | Computational sequence read from file D:\cmumosei\CMU_MOSEI_TimestampedWordVectors.csd ...
[2021-02-26 17:00:03.358] | Warning | Validation of the computational sequence skipped by user request.
[2021-02-26 17:00:03.358] | Success | Dataset initialized successfully ... 
[2021-02-26 17:00:03.373] | Success | Computational sequence read from file D:\cmumosei\CMU_MOSEI_VisualFacet42.csd ...
[2021-02-26 17:00:43.048] | Warning | Validation of the computational sequence skipped by user request.
[2021-02-26 17:00:43.048] | Success | Computational sequence read from file D:\cmumosei\CMU_MOSEI_VisualOpenFace2.csd ...
[2021-02-26 17:01:13.225] | Warning | Validation of the computational sequence skipped by user request.
[2021-02-26 17:01:13.22

In [9]:
[COVAREP, WordVec] = [basic_dataset.computational_sequences['COVAREP'],
                        basic_dataset.computational_sequences['WordVec']]
label = label_dataset.computational_sequences['mylabels']
OpenFace = second_dataset.computational_sequences['OpenFace']

In [16]:
real_length = []
list_label = list(label.keys())
pointer = 0
for l_l in list_label:
    if l_l in DATASET.standard_folds.standard_train_fold:
        real_label = label[l_l]['features'][0,1:]
        poria_label = emo_train_label[pointer][0]
        senti_train_label
        #if real_label == poria_label:
        #    print(True)
        print(real_label, poria_label)
    
        break
    

[0.6666667 0.6666667 0.        0.        0.        0.6666667] [0.         0.         0.33333333 0.66666667 0.66666667 0.        ]
